In [2]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import sqlite3
import os
import numpy as np
from requests_html import HTMLSession
from requests_file import FileAdapter

session = HTMLSession()

session.mount('file://',FileAdapter())
pwd = os.getcwd().replace("\\","/")
html_obj = session.get(f'file:///{pwd}/台股公告 _ Anue鉅亨.html')

soup = BeautifulSoup(html_obj.text,"html.parser")

In [3]:
titletime = []
id = []
idBefore = []
content = []
  

# # 日期============================
times = soup.select('._67tN')
for time in times:
  head,mid,tail = time.find('time')['datetime'].partition('T') # partition‘T'之後分為頭部，中部，尾部三個
  titletime.append(head)
# print(len(titletime))

# # 內容============================
# titles = soup.select('._1xc2')
# for title in titles:
#   head,mid,tail = title.find('h3').text.partition(':')
#   id.append(head)
#   content.append(title.text)
# # print(len(id))
# # print(len(content))

# 內容===============================================================================================================
titles = soup.select('._1xc2')
# 1.去除特殊字符
for title in titles:
  head,mid,tail = title.find('h3').text.partition(':')
  if head[0] == '「':
    head2 = head.replace('「','') # replace（代替），將字符替換成
    head3 = head2.find('、') # 在第幾位

    if head3 < 0: # 沒有、號
      head2 = head2.split('」',1) # split（切割），切割為【」以前的內容】和【」以後的內容】
      head2 = head2[0]
      idBefore.append(head2)
    else: # 有、號
      head2 = head2[:head3]
      head2 = head2.split('」',1) 
      head2 = head2[0]
      idBefore.append(head2)
  else:
    idBefore.append(head)
  content.append(title.text)
# 2.去除【月營收】
for x in idBefore:
  x2 = x.find('月營收')
  if x2 < 0: # 沒有【月營收】
    id.append(x)
  else: # 有【月營收】
    x3 = x2 - 1
    x4 = x[:x3]
    id.append(x4)
# =================================================================================================================


data = {
  'date': titletime,
  'code': id,
  'content': content
}


df = pd.DataFrame(data)
# print(df)

# # 今日新闻
newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期
df_mask = df['date']=='2022-09-06'            # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df.iloc[positions]
# print(filtered_df)


# connecting to sqlite
conn = sqlite3.connect('anuenews.db')
# cursor object
cursor = conn.cursor()
uniqueSql = '''select * from news t'''


for index, row in filtered_df.iterrows():
  try:
    cursor.execute(
        """INSERT OR IGNORE INTO news 
          (date, code, content)
          values(?,?,?)""",
          (row['date'],
          row['code'],
          row['content']),
        )
    conn.commit()
  except:
    pass

# print(filtered_df)

# 確認當日新聞 ========================================================================
dfToday = pd.read_sql(uniqueSql,conn)
# df_maskToday = dfToday['date']==newsToday            # 日期為今天的存檔
df_maskToday = dfToday['date']=='2022-09-06'           # 日期為今天的存檔
positionsToday = np.flatnonzero(df_maskToday)
filtered_dfToday=dfToday.iloc[positionsToday]
# ======================================================================================

filtered_dfToday

,date,code,content
2317,2022-09-06,志信,志信:志信國際股份有限公司（公司代號：2611）111年除權配股股票上市掛牌日期。
2318,2022-09-06,中信金,中信金:本公司將於111年9月19日支付一○九年度第二次無擔保普通公司債D券第二次利息公告
2319,2022-09-06,中信金,中信金:本公司將於111年9月19日支付一○九年度第二次無擔保普通公司債B券第二次利息公告
2320,2022-09-06,中信金,中信金:本公司將於111年9月19日支付一○九年度第二次無擔保普通公司債C券第二次利息公告
2321,2022-09-06,中信金,中信金:本公司將於111年9月19日支付一○九年度第二次無擔保普通公司債A券第二次利息公告
...,...,...,...
2568,2022-09-06,森鉅,森鉅:代子公司公告-新創環保公司取得固定資產事業廢棄物焚化暨污染防治系統工程
2569,2022-09-06,醣聯,醣聯:本公司有價證券近期多次達公佈注意交易資訊標準故公告相關訊息，以利投資人區別瞭解。
2570,2022-09-06,神基,神基:新聞稿-神基八月自結合併營收為29.37億元，年增8.34%
2571,2022-09-06,嘉澤,嘉澤:本公司受邀參加2022年元大證券第三季投資產業論壇


In [1]:
# 创建资料库 ====================================================================
# import sqlite3


# # connecting to sqlite
# conn = sqlite3.connect('anuenews.db')
# # cursor object
# cursor = conn.cursor()

# # Creating table
# news = """ CREATE TABLE news (
#     date DATE,
#     code TEXT,
#     content TEXT UNIQUE, 
#     UNIQUE(content)
# ); """

# cursor.execute(news)
# ================================================================================